In [1]:
import numpy as np
import pandas as pd
import math

from collections import Counter
from collections import OrderedDict

In [2]:
test_dict = {"<10": 0.08, "10-15": 0.27, "15-20": 0.10,
             "20-25":0.11,"25-30":0.15,"30-35":0.20,"35-37":0.07,
             "37+":0.02}
n = 4989

### Question 1

In [3]:
def truckSim(n,d):
    """
    This function takes a number, n, of trucks to simulate and a dictionary, d, of value-probability pairs and outputs
    a list of n different truck levels randomly simulated.  To do this, we generate a random number between 0 and 1
    then loop through each key-value pair, adding the probability to a counter.  Once the counter is greater than the 
    generated random number, we append the key (truck level) to the list to be output.
    """
    truck_levels = [] 
    for j in range(n): 
        cumulative_prob = 0
        random_num = np.random.uniform(0,1) 
        for (k,v) in d.items():
            cumulative_prob += v 
            if cumulative_prob > random_num: 
                truck_levels.append(k)
                break 
    return truck_levels

### Question 2

In [70]:
simlist = truckSim(1000,test_dict)

In [71]:
def truck1CI(alpha,keyname,n,simlist):
    finallist = []
    deltalist = []
    for i in range(n):
        templist = np.random.choice(simlist,len(simlist))
        pos = Counter(templist)[keyname]
        finallist.append(1.0*pos/len(templist))
    samplemean = 1.0*Counter(simlist)[keyname]/len(simlist)
    stderror = np.std(finallist)/np.sqrt(n)    
    deltalist = [fill - samplemean for fill in finallist]
    deltalist = sorted(deltalist,reverse=False)
    ub = deltalist[int(alpha*n)]
    lb = deltalist[int((1-alpha)*n)]
    return pd.Series([samplemean,samplemean+lb,samplemean+ub,stderror],index = ['Est.Value','CI Lower Bound','CI Upper Bound','Standard Error'])


In [72]:
df = pd.DataFrame([[1-0.1, '30-35', 100],[1-0.1, '30-35', 1000],[1-0.1, '30-35', 2500],
                   [1-0.1, '30-35', 5000],[1-0.1, '<10', 100],[1-0.1, '<10', 1000],
                   [1-0.1, '<10', 2500],[1-0.1, '<10', 5000]],
                  columns=['Confid.Level','Truck Fill Level','No of Bootstraps'])
df['Est.Value'],df['CI Lower Bound'],df['CI Upper Bound']= np.zeros(8),np.zeros(8),np.zeros(8)
df2= df.apply(lambda x:(truck1CI(x[0],x[1],x[2],simlist)),axis=1)
df['Est.Value'] = df2['Est.Value']
df['CI Lower Bound']= df2['CI Lower Bound']
df['CI Upper Bound']= df2['CI Upper Bound']
df['Standard Error']= df2['Standard Error']

In [73]:
df

,Confid.Level,Truck Fill Level,No of Bootstraps,Est.Value,CI Lower Bound,CI Upper Bound,Standard Error
0,0.9,30-35,100,0.222,0.203,0.235,0.001197
1,0.9,30-35,1000,0.222,0.204,0.239,0.000434
2,0.9,30-35,2500,0.222,0.205,0.238,0.000257
3,0.9,30-35,5000,0.222,0.205,0.239,0.000185
4,0.9,<10,100,0.073,0.062,0.083,0.000711
5,0.9,<10,1000,0.073,0.062,0.084,0.000269
6,0.9,<10,2500,0.073,0.062,0.083,0.000163
7,0.9,<10,5000,0.073,0.062,0.084,0.000117


### Question 3

In [67]:
simlist = truckSim(4989,test_dict)

In [68]:
dfQ3 = pd.DataFrame([[1-0.05, '35-37', 200],[1-0.05, '37+', 200],[1-0.1, '35-37', 200],
                   [1-0.1, '37+', 200]],
                  columns=['Confid.Level','Truck Fill Level','No of Bootstraps'])
dfQ3['Est.Value'],dfQ3['CI Lower Bound'],dfQ3['CI Upper Bound']= np.zeros(4),np.zeros(4),np.zeros(4)
df2 = dfQ3.apply(lambda x:(truck1CI(x[0],x[1],x[2],simlist)),axis=1)
dfQ3['Est.Value'] = df2['Est.Value']
dfQ3['CI Lower Bound']= df2['CI Lower Bound']
dfQ3['CI Upper Bound']= df2['CI Upper Bound']
dfQ3['Standard Error']= df2['Standard Error']

In [69]:
dfQ3

,Confid.Level,Truck Fill Level,No of Bootstraps,Est.Value,CI Lower Bound,CI Upper Bound,Standard Error
0,0.95,35-37,200,0.069553,0.063339,0.076368,0.000267
1,0.95,37+,200,0.018040,0.015033,0.021447,0.000134
2,0.90,35-37,200,0.069553,0.064542,0.073762,0.000250
3,0.90,37+,200,0.018040,0.015634,0.020846,0.000142


### Question 4

In [22]:
pos = 0
for i in range(1000):
    result_list = truckSim(n, test_dict)
    if result_list.count('35-37') >= 368:
        pos = pos + 1s

In [23]:
print('Estimated probability of 35-37ft >= 368:')
print(1.0*pos/1000)

Estimated probability of 35-37ft >= 368:
0.177


In [24]:
pos = 0
for i in range(1000):
    result_list = truckSim(n, test_dict)
    if result_list.count('37+') >= 108:
        pos = pos + 1

In [25]:
print('Estimated probability of 37+ >= 108:')
print(1.0*pos/1000)

Estimated probability of 37+ >= 108:
0.212


### Question 5

In [26]:
pos = 0
for i in range(1000):
    result_list = truckSim(n, test_dict)
    if result_list.count('37+') >= 108 and result_list.count('35-37') >= 368:
        pos = pos + 1

In [27]:
print('Estimated probability of 37+ >= 108 AND 35-37 >= 368:')
print(1.0*pos/1000)

Estimated probability of 37+ >= 108 AND 35-37 >= 368:
0.026
